# Analytics with Synapse Link and Delta Lake

This notebook covers setting up and using analytics capabilities:

- **Azure Synapse Link for Cosmos DB** - Real-time analytics without ETL
- **Delta Lake Medallion Architecture** - Bronze/Silver data layers
- **SQL Serverless Queries** - Query data with T-SQL

## Configuration

Configure your Azure resources below. Resources can be in **different resource groups** - specify the appropriate resource group for each service.

In [ ]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

$SUBSCRIPTION_ID = "363ef5d1-0e77-4594-a530-f51af23dbf8c"

# Resource Groups (can be different for each resource)
$COSMOS_RESOURCE_GROUP = "rg-dlz-cosmosdb-east2-sandbox"
$SYNAPSE_RESOURCE_GROUP = "rg-sandbox-demo-east2"
$STORAGE_RESOURCE_GROUP = "rg-dlz-aiml-stack-dev"

# Cosmos DB
$COSMOS_ACCOUNT = "cosmosdb-dlz-east2-sandbox"
$COSMOS_DATABASE = "DocumentsDB"
$COSMOS_CONTAINER = "ExtractedDocuments"

# Synapse Analytics
$SYNAPSE_WORKSPACE = "synapse-sandbox-east2-dlz"

# Storage
$STORAGE_ACCOUNT = "aimldatastore"

# ============================================

az account set --subscription $SUBSCRIPTION_ID

Write-Host "Configuration set" -ForegroundColor Green
Write-Host ""
Write-Host "Resource Groups:" -ForegroundColor Cyan
Write-Host "  Cosmos DB: $COSMOS_RESOURCE_GROUP"
Write-Host "  Synapse:   $SYNAPSE_RESOURCE_GROUP"
Write-Host "  Storage:   $STORAGE_RESOURCE_GROUP"

In [ ]:
# Validate resource access across resource groups
Write-Host "Validating resource access..." -ForegroundColor Cyan
$errors = @()

# Check Cosmos DB
Write-Host "`nChecking Cosmos DB..." -ForegroundColor Gray
$cosmosCheck = az cosmosdb show --name $COSMOS_ACCOUNT --resource-group $COSMOS_RESOURCE_GROUP --query "name" -o tsv 2>&1
if ($LASTEXITCODE -eq 0) {
    Write-Host "  ✓ Cosmos DB: $COSMOS_ACCOUNT" -ForegroundColor Green
} else {
    Write-Host "  ✗ Cosmos DB: $COSMOS_ACCOUNT not found in $COSMOS_RESOURCE_GROUP" -ForegroundColor Red
    $errors += "Cosmos DB"
}

# Check Synapse Workspace
Write-Host "Checking Synapse Workspace..." -ForegroundColor Gray
$synapseCheck = az synapse workspace show --name $SYNAPSE_WORKSPACE --resource-group $SYNAPSE_RESOURCE_GROUP --query "name" -o tsv 2>&1
if ($LASTEXITCODE -eq 0) {
    Write-Host "  ✓ Synapse: $SYNAPSE_WORKSPACE" -ForegroundColor Green
} else {
    Write-Host "  ✗ Synapse: $SYNAPSE_WORKSPACE not found in $SYNAPSE_RESOURCE_GROUP" -ForegroundColor Red
    $errors += "Synapse"
}

# Check Storage Account
Write-Host "Checking Storage Account..." -ForegroundColor Gray
$storageCheck = az storage account show --name $STORAGE_ACCOUNT --resource-group $STORAGE_RESOURCE_GROUP --query "name" -o tsv 2>&1
if ($LASTEXITCODE -eq 0) {
    Write-Host "  ✓ Storage: $STORAGE_ACCOUNT" -ForegroundColor Green
} else {
    Write-Host "  ✗ Storage: $STORAGE_ACCOUNT not found in $STORAGE_RESOURCE_GROUP" -ForegroundColor Red
    $errors += "Storage"
}

# Summary
if ($errors.Count -eq 0) {
    Write-Host "`n✓ All resources validated successfully" -ForegroundColor Green
} else {
    Write-Host "`n✗ Failed to access: $($errors -join ', ')" -ForegroundColor Red
    Write-Host "Verify resource names and resource groups are correct" -ForegroundColor Yellow
}

## 1. Azure Synapse Link for Cosmos DB

Synapse Link enables hybrid transactional and analytical processing (HTAP) by automatically syncing operational data to an analytical store.

### Benefits
- **No ETL Required** - Data automatically syncs (2-5 minute latency)
- **No Impact on Transactions** - Analytics run against analytical store
- **Cost-Effective** - Pay only for analytical storage consumed

In [ ]:
# Check if Synapse Link is enabled on Cosmos DB account
Write-Host "Checking Synapse Link status..." -ForegroundColor Cyan

$cosmosAccount = az cosmosdb show `
    --name $COSMOS_ACCOUNT `
    --resource-group $COSMOS_RESOURCE_GROUP `
    --query "{Name:name, SynapseLink:enableAnalyticalStorage}" `
    --output json | ConvertFrom-Json

Write-Host "`nCosmos DB Account: $($cosmosAccount.Name)"
Write-Host "Resource Group: $COSMOS_RESOURCE_GROUP"

if ($cosmosAccount.SynapseLink -eq $true) {
    Write-Host "Synapse Link: ENABLED" -ForegroundColor Green
} else {
    Write-Host "Synapse Link: DISABLED" -ForegroundColor Yellow
    Write-Host "Enable via Azure Portal: Cosmos DB > Azure Synapse Link > Enable" -ForegroundColor Yellow
}

In [ ]:
# Check container analytical store settings
Write-Host "Checking container analytical store..." -ForegroundColor Cyan

$container = az cosmosdb sql container show `
    --account-name $COSMOS_ACCOUNT `
    --database-name $COSMOS_DATABASE `
    --name $COSMOS_CONTAINER `
    --resource-group $COSMOS_RESOURCE_GROUP `
    --query "resource.analyticalStorageTtl" `
    --output tsv

if ($container -eq "-1") {
    Write-Host "Analytical Store: ENABLED (infinite retention)" -ForegroundColor Green
} elseif ($container -gt 0) {
    Write-Host "Analytical Store: ENABLED (TTL: $container seconds)" -ForegroundColor Green
} else {
    Write-Host "Analytical Store: DISABLED" -ForegroundColor Yellow
}

In [ ]:
# Enable Synapse Link on Cosmos DB account (if not enabled)
Write-Host "Enabling Synapse Link on Cosmos DB account..." -ForegroundColor Cyan
Write-Host "NOTE: This operation cannot be reversed!" -ForegroundColor Yellow

# Uncomment to enable:
az cosmosdb update `
    --name $COSMOS_ACCOUNT `
    --resource-group $COSMOS_RESOURCE_GROUP `
    --enable-analytical-storage true

Write-Host "Uncomment the command above to enable Synapse Link" -ForegroundColor Yellow

## 2. Synapse Spark Notebooks

Query Cosmos DB analytical store using Spark in Synapse Studio.

In [ ]:
# List Synapse notebooks
Write-Host "Synapse notebooks in workspace:" -ForegroundColor Cyan

az synapse notebook list `
    --workspace-name $SYNAPSE_WORKSPACE `
    --query "[].name" `
    --output table

### Sample Spark Code for Synapse Notebooks

Use this code in Synapse Studio Spark notebooks:

```python
# Read from Cosmos DB analytical store
df = spark.read \
    .format("cosmos.olap") \
    .option("spark.synapse.linkedService", "LS_CosmosDB") \
    .option("spark.cosmos.container", "ExtractedDocuments") \
    .load()

# Run analytics
df.createOrReplaceTempView("documents")

# Count by model
spark.sql("""
    SELECT modelId, COUNT(*) as count, AVG(modelConfidence) as avg_conf
    FROM documents
    GROUP BY modelId
""").show()

# Recent processing activity
spark.sql("""
    SELECT sourceFile, processedAt, status, modelConfidence
    FROM documents
    WHERE processedAt > date_sub(current_timestamp(), 7)
    ORDER BY processedAt DESC
    LIMIT 20
""").show()
```

## 3. Delta Lake Medallion Architecture

Transform data from Cosmos DB into Delta Lake layers:

| Layer | Description | Location |
|-------|-------------|----------|
| **Bronze** | Raw data with ingestion metadata | `delta/bronze/extracted_documents/` |
| **Silver** | Cleaned, flattened, partitioned data | `delta/silver/documents/` |

### Sample Bronze Layer Code

```python
from datetime import datetime
from pyspark.sql.functions import current_timestamp, lit

# Read from Cosmos DB analytical store
df_cosmos = spark.read \
    .format("cosmos.olap") \
    .option("spark.synapse.linkedService", "LS_CosmosDB") \
    .option("spark.cosmos.container", "ExtractedDocuments") \
    .load()

# Add ingestion metadata
df_bronze = df_cosmos \
    .withColumn("_ingested_at", current_timestamp()) \
    .withColumn("_source", lit("cosmos_analytical_store"))

# Write to Bronze layer
bronze_path = f"abfss://delta@{storage_account}.dfs.core.windows.net/bronze/extracted_documents/"

df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .save(bronze_path)
```

### Sample Silver Layer Code

```python
from pyspark.sql.functions import col, to_timestamp, explode

# Read Bronze layer
df_bronze = spark.read.format("delta").load(bronze_path)

# Transform to Silver
df_silver = df_bronze \
    .select(
        col("id").alias("document_id"),
        col("sourceFile").alias("source_file"),
        to_timestamp(col("processedAt")).alias("processed_at"),
        col("status"),
        col("modelId").alias("model_id"),
        col("modelConfidence").alias("model_confidence"),
        col("docType").alias("document_type"),
        col("fields"),
        col("_ingested_at")
    ) \
    .withColumn("_is_valid", col("status") == "completed")

# Write to Silver layer with partitioning
silver_path = f"abfss://delta@{storage_account}.dfs.core.windows.net/silver/documents/"

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("status", "document_type") \
    .save(silver_path)
```

## 4. SQL Serverless Queries

Query data using T-SQL without provisioning dedicated resources.

In [ ]:
# List SQL scripts in workspace
Write-Host "SQL scripts in workspace:" -ForegroundColor Cyan

az synapse sql-script list `
    --workspace-name $SYNAPSE_WORKSPACE `
    --query "[].name" `
    --output table

### Query Cosmos DB via SQL Serverless

```sql
-- Query Cosmos DB analytical store directly
SELECT TOP 100
    doc.id,
    doc.sourceFile,
    doc.processedAt,
    doc.status,
    doc.modelId,
    doc.modelConfidence
FROM OPENROWSET(
    'CosmosDB',
    'Account=your-cosmos-account;Database=DocumentsDB;Key=your-key',
    ExtractedDocuments
) WITH (
    id VARCHAR(100),
    sourceFile VARCHAR(500),
    processedAt VARCHAR(50),
    status VARCHAR(50),
    modelId VARCHAR(100),
    modelConfidence FLOAT
) AS doc
ORDER BY doc.processedAt DESC;
```

### Query Delta Lake via SQL Serverless

```sql
-- Query Silver Delta Lake layer
SELECT 
    status,
    document_type,
    COUNT(*) as document_count,
    AVG(model_confidence) as avg_confidence
FROM OPENROWSET(
    BULK 'https://your-storage.dfs.core.windows.net/delta/silver/documents/',
    FORMAT = 'DELTA'
) AS docs
GROUP BY status, document_type
ORDER BY document_count DESC;

-- Time-based analysis
SELECT 
    CAST(processed_at AS DATE) as process_date,
    COUNT(*) as documents_processed,
    SUM(CASE WHEN _is_valid = 1 THEN 1 ELSE 0 END) as successful
FROM OPENROWSET(
    BULK 'https://your-storage.dfs.core.windows.net/delta/silver/documents/',
    FORMAT = 'DELTA'
) AS docs
WHERE processed_at > DATEADD(day, -30, GETDATE())
GROUP BY CAST(processed_at AS DATE)
ORDER BY process_date DESC;
```

## 5. Deploy Analytics Artifacts

Deploy notebooks and SQL scripts to Synapse workspace.

In [ ]:
# Deploy all Synapse artifacts including notebooks and SQL scripts

# Get the notebook directory and navigate to project root
$notebookDir = (Get-Location).Path
if ($notebookDir -match "notebooks$") {
    $projectRoot = Split-Path $notebookDir -Parent
} else {
    # Assume we're already at project root or find it
    $projectRoot = $notebookDir
    while ($projectRoot -and -not (Test-Path (Join-Path $projectRoot "scripts"))) {
        $projectRoot = Split-Path $projectRoot -Parent
    }
}

$STORAGE_URL = "https://$STORAGE_ACCOUNT.blob.core.windows.net"
$COSMOS_ENDPOINT = "https://$COSMOS_ACCOUNT.documents.azure.com:443/"

Write-Host "Deploying Synapse analytics artifacts..." -ForegroundColor Cyan
Write-Host "  Synapse Workspace: $SYNAPSE_WORKSPACE (RG: $SYNAPSE_RESOURCE_GROUP)" -ForegroundColor Gray
Write-Host "  Storage Account:   $STORAGE_ACCOUNT (RG: $STORAGE_RESOURCE_GROUP)" -ForegroundColor Gray
Write-Host "  Cosmos DB Account: $COSMOS_ACCOUNT (RG: $COSMOS_RESOURCE_GROUP)" -ForegroundColor Gray

$scriptPath = Join-Path $projectRoot "scripts\Deploy-SynapseArtifacts.ps1"

if (Test-Path $scriptPath) {
    & $scriptPath `
        -WorkspaceName $SYNAPSE_WORKSPACE `
        -ResourceGroup $SYNAPSE_RESOURCE_GROUP `
        -DeploymentMode direct `
        -StorageAccountUrl $STORAGE_URL `
        -CosmosEndpoint $COSMOS_ENDPOINT
} else {
    Write-Host "Script not found at: $scriptPath" -ForegroundColor Red
    Write-Host "Run this from the project root or ensure scripts/Deploy-SynapseArtifacts.ps1 exists" -ForegroundColor Yellow
}

## KQL Queries for Analytics

Use KQL (Kusto Query Language) for advanced analytics in Synapse.

### Sample KQL Query

Run this in Synapse Studio KQL notebook (connected to your Data Explorer pool or external cluster):

```kql
// Document processing summary
documents
| summarize 
    TotalDocuments = count(),
    SuccessCount = countif(status == "completed"),
    FailureCount = countif(status == "failed"),
    AvgConfidence = avg(modelConfidence)
| extend SuccessRate = round(100.0 * SuccessCount / TotalDocuments, 2)
```

## Next Steps

1. **Open Synapse Studio** - Access notebooks and SQL scripts
2. **Create Spark Pool** - Required for running Spark notebooks
3. **Set up monitoring** - See `07-Monitoring-Troubleshooting.ipynb`

### Synapse Studio Links

Open your Synapse Studio at: `https://web.azuresynapse.net/?workspace=%2fsubscriptions%2f{subscription_id}%2fresourceGroups%2f{resource_group}%2fproviders%2fMicrosoft.Synapse%2fworkspaces%2f{workspace_name}`